In [5]:
#### Importing tools that I need

import os, datetime, csv, pdb, time, multiprocessing
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pylab as plt
import plotly.plotly as py
import plotly.tools as tls
import cufflinks as cf
import re
from pandas import DataFrame
from collections import Counter
from joblib import Parallel, delayed
from joblib.pool import has_shareable_memory

py.sign_in('1IfByLAN2IfByC','r50yawfp79')


In [12]:
#### Dynamic Variables
#### These are what I have to change, think configuration settings
#### To search just one phone number, input number after /baskup-master/...

folder = '/Users/michael/Desktop/FOR MIKE/baskup-master/'
fillist = os.listdir(folder)


## to look for just one person, set JUSTNUM to True 
## to look for everyone, set JUSTNUM to False
## AND fill in a phonenumber in form '15124618901'
JUSTNUM = True

## put the 1 and dont put no spaces of any of that snooty shit
PHONENUMBER = '15124618901'

YOURPHONENUMBER = '15124618901'

## how far back do you want to go?
## to go back for all of time, set days back to 0
days_back = 365

## how many words are the "top words"?
topWordCount = 100

In [73]:
#### Go through each sub-folder and folder directory (where folder is defined above ) and read all texts into a list of strings. 


# mac to unix time delta (functions that turns strings into date/time)
mac2unix = 978304445
date =lambda tt: datetime.datetime.fromtimestamp(tt).strftime('%m-%d-%Y ')
time = lambda tt: datetime.datetime.fromtimestamp(tt).strftime('%H:%M:%S')
dateAndTime = lambda tt: datetime.datetime.fromtimestamp(tt).strftime('%m-%d-%Y %H:%M:%S')

#initializing (making boxes and naming them)
cleanTextList = []    
parsedTexts = []
i=0

# for all folder in the master dir (fillist)
for i in range(len(fillist)):
    os.chdir(folder)
    
    # go into folder 
    if os.path.isdir(fillist[i]):
        os.chdir(folder+fillist[i])
        textfiles = os.listdir(os.getcwd())
        SENDING_NUMBER = ''.join([s for s in fillist[i] if s.isdigit()])
        # jj is another term for index in this case
        for jj in range(len(textfiles)):
            if textfiles[jj].endswith('.txt'):
                with open(textfiles[jj]) as f:
                    text = f.readlines()
                    
                #Line (string)
                #1 means me, 0 means other person
                for line in text:
                    if line[0] == '1':
                        line = YOURPHONENUMBER + line[1:]

                    elif line[0] =='0':
                        line = SENDING_NUMBER + line[1:]

                    cleanTextList.append(line)
                   
                    # split a string into a list using our ||| delimiter
                    newLine = line.split('|||')

                    # error checking, get rid of new lines made in text messages
                    if newLine[0].isdigit() == True:
                   
                        # convert to real times
                        # date sent, date, received, sent
                        newLine[1] = dateAndTime(int(newLine[1])+mac2unix)

                        if int(newLine[2]) != 0:
                            newLine[2] = dateAndTime(int(newLine[2]) + mac2unix)

                        if int(newLine[3]) != 0:
                            newLine[3] = dateAndTime(int(newLine[3])+mac2unix)

                        ## turn into csv where each row represents a message                           
                        parsedTexts.append(newLine)


                    else: # its some fuckshit and we gonna delete it
                        del newLine
            
## write all text to csv              
with open(folder+'output.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(parsedTexts)           
        

In [25]:
#### here we turn our list of text messages and dates etc. into a more readable format (data frame)

headers = ['Sender', 'Date', 'Date Received', 'DateRead', 'Text']
df = DataFrame(parsedTexts, columns=headers)

# turn all the dates we parsed from strings into date+time
strp = lambda date: datetime.datetime.strptime(date, '%m-%d-%Y %H:%M:%S')
df['Date'] = [strp(d) for d in df['Date']]

# make a column with just the day so we can compare easier
df['Just Date'] = [d.date() for d in df['Date']]

# "" with hours
df['Just Hour'] = [d.hour for d in df['Date']]



## if JUSTNUM (only looking for a specific phone number) is true, 
## then the new dataframe is equal to the old dataframe 
## but only where the sender was equal to the phonenumber of interest
if JUSTNUM == True:
    df = df[df['Sender'] == PHONENUMBER]
            
# function that takes in a unsorted dataframe
# and organizes it into 24 periods to count and sort
# text messages by date 
def volumeTextHour(df, daysBack):
   
    # first we collect all the date ranges
    # slice the index to get data only from specified range
    df = df.sort(columns='Just Date', ascending=True)
  
    # your new dataframe is equal to your old dataframe but only where your date is greater than today minus 365
    if daysBack != 0:
        df = df[df['Date'] > (datetime.datetime.now() - datetime.timedelta(days=365))]
        
    # remove duplicates 
    hours = [df['Date'].iloc[d].hour for d in range(len(df))]
    sortedHours = list(set(hours)) 
    
    # make an array so we can count, initializing dataframe 
    mostFrequentHour = np.zeros((len(sortedHours), 3))
    mostFrequentHour = DataFrame(mostFrequentHour, columns=['Hour', 'Count', 'Messages'])

    # make first row
    for i in range(len(sortedHours)):
        mostFrequentHour['Hour'].iloc[i] = sortedHours[i]
        
        # loop through all rows in the original dataframe 
        # if the date matches the sorted date i, append 
        # the text message to a the list for that day
        message = [df['Text'].iloc[d] for d in range(len(df)) if df['Just Hour'].iloc[d] == sortedHours[i]]
        mostFrequentHour['Messages'].iloc[i] = message

        # find the number of texts for that day
        mostFrequentHour['Count'].iloc[i] = len(mostFrequentHour['Messages'].iloc[i])
  

    print "finished Hours"
    return mostFrequentHour


#### same thing as above but for date
def volumeTextDay(df, daysBack):
     # slice the index to get data only from specified range
    df = df.sort(columns='Just Date', ascending=True)
    if daysBack != 0:
        df = df[df['Date'] > (datetime.datetime.now() - datetime.timedelta(days=365))]
    
    dates = [df['Date'].iloc[d].date() for d in range(len(df))]
    sortedDates = list(set(dates))
    
    # make an array so we can count 
    mostFrequentDay = np.zeros((len(sortedDates), 3))
    mostFrequentDay = DataFrame(mostFrequentDay, columns=['Date', 'Count', 'Messages'])

   
    for i in range(len(sortedDates)):
        mostFrequentDay['Date'].iloc[i] = sortedDates[i]
        
        # loop through all rows in the original dataframe 
        # if the date matches the sorted date i, append 
        # the text message to a the list for that day
        message = [df['Text'].iloc[d] for d in range(len(df)) if df['Just Date'].iloc[d] == sortedDates[i]]
        mostFrequentDay['Messages'].iloc[i] = message

        # find the number of texts for that day
        mostFrequentDay['Count'].iloc[i] = len(mostFrequentDay['Messages'].iloc[i]) 
      
    print "finished days"
    return mostFrequentDay


#### goes through finds all words used, then searches all text messages and counts the number of 
#### and counts the number of times each word was used
def wordCounts(df, daysBack):
    
    ## slice the index to get data only from specified range
    df = df.sort(columns='Just Date', ascending=True)
    if daysBack != 0:
        df = df[df['Date'] > (datetime.datetime.now() - datetime.timedelta(days=365))]
    
    ## joining all values inside lists into one string
    ## find each unique word and make it a corpus (dictionary)
    ## corpus has each unique word
    ## export final dataframe with each unique word (for the count)
    word = ';'.join(df['Text']).lower()
    corpus = re.compile('\w+').findall(word)
    corpus = list(set(corpus))
    word_dataframe = np.zeros((len(corpus), 2))
    word_dataframe = DataFrame(word_dataframe, columns=['Word', 'Count'])
    word_dataframe['Word'] = corpus

    
    print "the total word count is: {total} \n the number of unique words is:".format(
    total=len(word), unique=len(corpus))
    ## output dataframe and string of all words
    return word_dataframe, word


#### for each row in wordDF (in this case a unique word used in conversations)
#### the count (number of times it was said) is equal to the sum of each unique word 
#### in wordString (long string of all texts)
def wordCountParFor(wordDF, wordString):
    for i in range(len(wordDF)):
        wordDF['Count'].iloc[i] = sum(1 for _ in \
            re.finditer(r'\b%s\b' % re.escape(wordDF['Word'].iloc[i]), wordString))
    ## export updated dataframe (with count)
    print "finished counting all the words"
    return wordDF

#### defining functions for regular expressions (natural language processing)
def topWordsFind(numWords, wordCount):
    wordCount = wordCount.sort(columns='Count',ascending=False)
    topWords = wordCount[0:numWords]
    return topWords

#### look for words, red section will have word you're looking for
#### good for words not phrases
def wordSearch(wordCount, word):
    print wordCount.loc[wordCount['Word'] == word]
    

In [262]:
#### using the functions defined above 

#use the function
mostFrequentDay = volumeTextDay(df, days_back)
#reset the index to date
mostFrequentDay = mostFrequentDay.set_index('Date')
#sort 
mostFrequentDay = mostFrequentDay.sort(ascending=False)

#use the function
mostFrequentHour = volumeTextHour(df, days_back)
#reset the index to date
mostFrequentHour = mostFrequentHour.set_index('Hour')
#sort
mostFrequentHour = mostFrequentHour.sort(ascending=False)

#use function
wordCount, wordString = wordCounts(df, days_back)
#split word count into 5 equal pieces (for parallel processing)
splitWordCount = np.array_split(wordCount, 5)

# use 5 cores with each searching one chunk of wordCount
result = Parallel(n_jobs=5)(delayed(wordCountParFor)(splitWordCount[i], wordString) for i in range(5))
wordCounts = pd.concat(result, axis=0)
wordCounts = wordCount.sort(columns='Count', ascending=False)

## write to csv
##file naming
if JUSTNUM == True:
    filename = folder+'/wordCount_'+PHONENUMBER+'.csv'
else:
    filename = folder+'/wordCount_everyone.csv'
wordCount.to_csv(filename)

#find top words and print done
topWord = topWordsFind(topWordCount, wordCounts)
print "finished making shit"

finished days
finished Hours
finished making shit
finished counting all the words
finished counting all the words
finished counting all the words
finished counting all the words
finished counting all the words


In [16]:
#### make plots with plotly plug in

## plot the most common words 
topWord.iplot(kind='scatter', x='Word', y='Count', mode='markers', world_readable=True)

NameError: name 'topWord' is not defined

In [288]:
#### make plots with plotly plug in

## plot the most frequent days
mostFrequentDay.iplot(world_readable=True)

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [289]:
#### make plots with plotly plug in

## plot the most frequent hour
mostFrequentHour.iplot(world_readable=True)

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [291]:
#### go through dataset and search for words (phrases wont work)

wordOfInterest = 'ok'
wordSearch(wordCounts, wordOfInterest)

     Word  Count
8781   ok    463


In [294]:
#### go through and search for phrases (only set up for all of time)
    # true means I said it
    # false means we all said it

sent = False
# word to look for
words = ['ok']
#words = ['duh', 'luv u', 'luv you']

count=0
for i in range(len(cleanTextList)):
    for word in words:
        if sent == True:
            if (word.lower() in cleanTextList[i].lower() and PHONENUMBER in cleanTextList[i]):
                count = count+1
        elif sent == False: 
            if (word.lower() in cleanTextList[i].lower()):
                count = count+1
                
            
print "{word} was said {count} times".format(word=word, count=count)

ok was said 7803 times


In [295]:
topWord = topWordsFind(topWordCount, wordCounts)
topWord

,Word,Count
4838,i,9378
3755,you,5335
1288,to,4977
5447,it,3116
2999,the,3088
3900,a,2512
3641,t,2488
3704,lol,2334
8510,and,2182
3431,that,2054


In [26]:
df

,Sender,Date,Date Received,DateRead,Text,Just Date,Just Hour
13,15124618901,2013-11-06 18:11:41,0,0,Lex I probably wouldn't smoke any weed if you ...,2013-11-06,18
15,15124618901,2013-11-06 18:26:37,0,0,Hahah okay! Yeah honestly i don't get why ppl ...,2013-11-06,18
17,15124618901,2013-11-06 18:28:45,0,0,Luuuucky you\n,2013-11-06,18
32,15124618901,2013-11-10 15:57:17,0,0,Awk I meant ur Insta post\n,2013-11-10,15
34,15124618901,2013-11-10 15:57:17,0,0,☺️\n,2013-11-10,15
43,15124618901,2013-11-12 13:34:21,0,0,Lololol not mine \n,2013-11-12,13
44,15124618901,2013-11-12 13:34:21,0,0,I'm about to come to trustee\n,2013-11-12,13
46,15124618901,2013-11-12 13:36:29,0,0,Hahaha good guess but not mine ... Does this m...,2013-11-12,13
49,15124618901,2013-11-13 11:28:29,0,0,Across frum u\n,2013-11-13,11
50,15124618901,2013-11-16 14:44:45,0,0,Alexandira \n,2013-11-16,14


In [32]:
#### to compare the number of the texts for a person
def timeDelay(df):
    delay = 
    datetime.timedelta(datetime.datetime.fromtimestamp(df.loc[150895].Date).strftime('%H:%M:%S'), \
                   datetime.datetime.fromtimestamp(df.loc[150895].DateRead).strftime( '%H:%M:%S') )

TypeError: a float is required

In [34]:
datetime.timedelta(df.loc[150895].Date,df.loc[150895].DateRead) 

TypeError: unsupported type for timedelta seconds component: str

In [40]:
datetime.datetime.strptime(df.loc[150895].DateRead, )

TypeError: strptime() takes exactly 2 arguments (1 given)

In [56]:
datetime.timedelta(datetime.datetime.strptime(df.loc[150895].DateRead, '%m-%d-%Y %H:%M:%S'), df.loc[150895].Date)

TypeError: unsupported type for timedelta seconds component: Timestamp

In [109]:
datetime.datetime.strptime(df.loc[150895].DateRead, '%m-%d-%Y %H:%M:%S')

datetime.datetime(2015, 2, 16, 12, 21, 49)

In [120]:
datetime.datetime.strptime(df.loc[150895].DateRead, '%m-%d-%Y %H:%M:%S')

datetime.datetime(2015, 2, 16, 12, 21, 49)

In [155]:
def timeDelay(df):
    delay = []
    for i in range(len(df['DateRead'])):
        try:
            int(df['DateRead'].loc[i])
        except ValueError:
            df['DateRead'].loc[i] = strp(df['DateRead'].loc[i])
    
        delay.append(datetime.timedelta(df['DateRead'].loc[i], df['Date'].loc[i]))

    

In [158]:
df['DateRead'].loc[150901]
       

'02-16-2015 12:34:37'

In [157]:
timeDelay(df)

KeyError: 'the label [0] is not in the [index]'